In [1]:
from load_sms_dataset import *

/Users/jason.xie/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [4]:
X_train, X_test, y_train, y_test, vectorizer = load_spam_dataset("~/Downloads/smsspamcollection/SMSSpamCollection")

In [82]:
class NaiveBayesClassifier:
    '''
        A binary classifier - P(y | x) = P(x | y) * P(y) / P(x)
    '''
   
    def fit(self, X, y, vocab=None):
        self.vocab = vocab
        self.priors = np.array([1 - np.mean(y), np.mean(y)])
        
        self.likelihood = np.zeros((X.shape[1], 2)) # P(x | y)
        for i in range(X.shape[1]): # go over every word
            arr = X[:,i].todense()
            for label in [0, 1]:
                arr_l = arr[y == label]
                self.likelihood[i, label] = np.mean(arr_l >= 1) # wow this is so efficient
    
    def predict(self, X):
        print X[0]
        print X[0].shape


In [81]:
clf = NaiveBayesClassifier()
clf.fit(X_train, y_train, vocab=vectorizer.vocabulary_inverted)

[[ 0.          0.01174168]
 [ 0.          0.03913894]
 [ 0.00031037  0.        ]
 [ 0.          0.00195695]
 [ 0.          0.00195695]
 [ 0.          0.00195695]
 [ 0.          0.00195695]
 [ 0.          0.00195695]
 [ 0.          0.        ]
 [ 0.          0.01369863]]
